## Initial Probe Exploration

### Goals:
- Train a simple linear logistic regression probe on Llama-3-7b
    - Update Alex 19/02/25: since I'm running this on my macbook, I'll just use the 1b model 
- Understand GPU capacity - can we do inference with 70B?
- Look at the probe activations / test set classifications

### Timeline:
- 18/02/25
- 19/02/25

In [1]:
# Imports
from models_under_pressure.probes import (
    train_single_layer,
    compute_accuracy,
    create_activations,
    LinearProbe,
)
import numpy as np

from transformers import AutoTokenizer, AutoModelForCausalLM
from joblib import Parallel, delayed
import os
import pandas as pd

/Users/Alex/Development/lasr/models-under-pressure/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Dataset

The dataset was generated using GPT-4o. It consists of 20 examples with red things and 20 examples with green things. We hope to learn a classifier / probe for green or red objects.

`Command: Generate 20 sentences about red things. Generate 20 sentences about green things. Put them in a JSON array of strings.`

In [2]:
df = pd.read_csv("dataset.csv")

X = df["prompt_text"].tolist()
y = df["high_stakes"].tolist()

# Set random seed for reproducibility
np.random.seed(42)

# Get total number of samples
n_samples = len(X)
print(n_samples)

# Generate random indices for train/test split (80/20)
indices = np.random.permutation(n_samples)
train_size = int(0.8 * n_samples)

# Split data into train and test sets
train_text = [X[i] for i in indices[:train_size]]
test_text = [X[i] for i in indices[train_size:]]
train_labels = [y[i] for i in indices[:train_size]]
test_labels = [y[i] for i in indices[train_size:]]


100


## Generate the Feature Inputs for the Probe

In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
model_name = "meta-llama/Llama-3.2-1B-Instruct"

# device = 'cuda:1'
device = "cpu"

# Load the LLaMA-3-1B model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Run the model on the train and test data, recording the activations

train_acts = create_activations(
    model=model, tokenizer=tokenizer, text=train_text, device=device
)

test_acts = create_activations(
    model=model, tokenizer=tokenizer, text=test_text, device=device
)


Layer: 0, Activation Shape: torch.Size([80, 34, 2048])
Layer: 1, Activation Shape: torch.Size([80, 34, 2048])
Layer: 2, Activation Shape: torch.Size([80, 34, 2048])
Layer: 3, Activation Shape: torch.Size([80, 34, 2048])
Layer: 4, Activation Shape: torch.Size([80, 34, 2048])
Layer: 5, Activation Shape: torch.Size([80, 34, 2048])
Layer: 6, Activation Shape: torch.Size([80, 34, 2048])
Layer: 7, Activation Shape: torch.Size([80, 34, 2048])
Layer: 8, Activation Shape: torch.Size([80, 34, 2048])
Layer: 9, Activation Shape: torch.Size([80, 34, 2048])
Layer: 10, Activation Shape: torch.Size([80, 34, 2048])
Layer: 11, Activation Shape: torch.Size([80, 34, 2048])
Layer: 12, Activation Shape: torch.Size([80, 34, 2048])
Layer: 13, Activation Shape: torch.Size([80, 34, 2048])
Layer: 14, Activation Shape: torch.Size([80, 34, 2048])
Layer: 15, Activation Shape: torch.Size([80, 34, 2048])
All activations shape: torch.Size([16, 80, 34, 2048])
Layer: 0, Activation Shape: torch.Size([20, 33, 2048])
Layer

## Training Code for the Probe

Use `sklearn` logistic regression classifier to learn a linear classifier on the activations from the model. We do the following:

1. Create the y labels (1 for red and 0 for green)
2. Restructure X to match sklearn (Batch_size, Embedd_dim) -> One per layer, final seq pos **TODO: Iterate in Future**  
3. Run Logistic Regression
4. Test on 5 test data points

In [5]:
model_params = {"C": 1, "random_state": 42, "fit_intercept": False}

layer_probes: list[LinearProbe] = Parallel(n_jobs=16)(
    delayed(train_single_layer)(acts, train_labels, model_params) for acts in train_acts
)  # type: ignore

accuracies = [
    compute_accuracy(probe, test_acts[i], test_labels)
    for i, probe in enumerate(layer_probes)
]

accuracies

[0.85,
 0.8,
 0.95,
 0.9,
 0.95,
 0.95,
 0.9,
 0.9,
 0.9,
 0.9,
 0.9,
 0.95,
 0.95,
 0.95,
 0.95,
 0.95]

In [ ]:
from sklearn.metrics import roc_auc_score

# Calculate AUC-ROC for each layer
auc_scores = []
for i, probe in enumerate(layer_probes):
    # Get probability predictions for positive class
    X = probe._preprocess_activations(test_acts[i])
    y_proba = probe._model.predict_proba(X)[:, 1]

    # Print debugging information
    print(f"\nLayer {i}:")
    print("True labels:", test_labels)
    print("Predicted probabilities:", y_proba.round(3))

    # Verify we have variation in both labels and predictions
    print(f"Unique true labels: {np.unique(test_labels)}")
    print(f"Prediction range: [{y_proba.min():.3f}, {y_proba.max():.8f}]")

    # Calculate AUC-ROC
    try:
        auc = roc_auc_score(test_labels, y_proba)
        auc_scores.append(auc)
        print(f"AUC-ROC: {auc:.3f}")
    except Exception as e:
        print(f"Error calculating AUC-ROC: {e}")
        auc_scores.append(None)



Layer 0:
True labels: [0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0]
Predicted probabilities: [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5]
Unique true labels: [0 1]
Prediction range: [0.500, 0.500]
AUC-ROC: 0.919

Layer 1:
True labels: [0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0]
Predicted probabilities: [0.5   0.501 0.5   0.5   0.5   0.501 0.5   0.5   0.5   0.5   0.5   0.5
 0.5   0.5   0.5   0.501 0.5   0.5   0.5   0.5  ]
Unique true labels: [0 1]
Prediction range: [0.500, 0.501]
AUC-ROC: 1.000

Layer 2:
True labels: [0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0]
Predicted probabilities: [0.512 0.512 0.512 0.512 0.512 0.512 0.512 0.512 0.512 0.512 0.512 0.512
 0.512 0.512 0.512 0.512 0.512 0.512 0.512 0.512]
Unique true labels: [0 1]
Prediction range: [0.512, 0.512]
AUC-ROC: 0.990

Layer 3:
True labels: [0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0]
Predicted probabilities: [0.512 0.512 0.

In [15]:
# Get predictions for layer 5
layer_5_probe = layer_probes[5]
X = layer_5_probe._preprocess_activations(test_acts[5])
y_proba = layer_5_probe._model.predict_proba(X)[:, 1]


results_df = pd.DataFrame(
    {"True Label": test_labels, "Predicted Probability": y_proba}
).round(3)

# Sort by predicted probability for easier analysis
results_df = results_df.sort_values("Predicted Probability", ascending=False)

# Display the results
print("\nLayer 5 Predictions vs True Labels:")
print(results_df)



Layer 5 Predictions vs True Labels:
    True Label  Predicted Probability
1            1                  0.513
7            1                  0.513
6            1                  0.513
5            1                  0.513
16           1                  0.513
17           1                  0.513
15           1                  0.513
18           1                  0.513
12           1                  0.513
3            1                  0.512
0            0                  0.512
2            0                  0.512
8            1                  0.512
19           0                  0.512
9            0                  0.511
4            0                  0.511
11           0                  0.511
10           0                  0.511
14           0                  0.511
13           0                  0.511
